In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pyepo
from pyepo.model.grb import optGrbModel
import torch
from torch import nn
from torch.utils.data import DataLoader
from gurobipy import Model, GRB, quicksum
from sklearn.preprocessing import StandardScaler
import pandas as pd
import wandb 
wandb.login()
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import time
from tqdm import tqdm
from pyepo.metric.regretParams import regretParams
# train model

#from sklearn_extra.cluster import KMedoids
import copy

Auto-Sklearn cannot be imported.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: frederikskoufertin (Pyepo_special). Use `wandb login --relogin` to force relogin


In [2]:
# load data
red = (0.77, 0, 0.05) # (196, 0, 13)
blue = (0.12, 0.24, 1) # (31, 61, 255)
# green = (0.31, 1, 0.34) # (79, 255, 87)
green = (0.122, 00.816, 0.51) # (31, 208, 130)
navyblue = (0, 0, 0.4) # (0, 0, 102)
black = (0, 0, 0)
white = (1, 1, 1)
cgreen = (0.57254902, 0.7254902 , 0.51372549) # (146, 185, 131)
cblue = (0.70196078, 0.83137255, 1) # (179, 212, 255)

top_domain = 53.32 # 90% quantile



def import_data(negative_prices=False):
    # import data and set constants
    all_data = pd.read_csv("2020_data.csv")
    prices_UP = np.maximum(all_data["UP"].to_numpy(),0)
    prices_DW = np.maximum(all_data["DW"].to_numpy(),0)
    prices_F = np.maximum(all_data["forward_RE"].to_numpy(),0)
    prices_forecast = np.maximum(all_data["forward_FC"].to_numpy(), 0)

    nominal_wind = 10
    features = all_data.loc[:, ["Offshore DK2", "Offshore DK1", "Onshore DK2", "Onshore DK1", "production_FC"]]
    features["forward"] = prices_F
    features_red = all_data.loc[:, ["production_FC"]]
    features_red["forward"] = prices_F
    realized = all_data.loc[:, "production_RE"].to_numpy()
    realized *= nominal_wind

    price_H = 35.2
    penalty = np.quantile(prices_UP, 0.95) # 95% quantile of deficit_settle price over all 2 years
    # penalty = 2 * price_H
    # penalty = np.max(prices_B) # Something HIGHER is needed apparently

    return (
        prices_UP,
        prices_DW,
        prices_F,
        prices_forecast,
        features,
        features_red,
        realized,
        price_H,
        penalty
    )

In [3]:
#Import data
(prices_UP,prices_DW,prices_F,prices_forecast,features,features_red,realized,price_H,penalty) = import_data()

# Change forward prices to forecast prices in features
features["forward"] = prices_forecast

periods = list(range(0, len(prices_F) )) # Total time considered 2020-2021
n_periods = 24 # Number of periods in a day
n_days = 50 # Number of days in training set and test set
n_hours = n_days * n_periods
# 4 without extra hydrogen 5 with extra hydrogen
num_cost = 5 # number of cost parameters
num_feat = n_periods*6 # size of feature
num_feat_rf = 2 # size of feature
num_item = num_cost*n_periods # number of predictions (Forward bid and Hydrogen)
num_feat = n_periods*6 # size of feature
#n_val_days = 10 # number of validation days 
#n_hours_val = n_periods*n_val_days
lambda_H_list = [price_H for i in range(n_periods)]
penalty_list = [-penalty for i in range(n_periods)]

lambda_H_list = [price_H for i in range(n_periods)]
penalty_list = [-penalty for i in range(n_periods)]

def flatten_extend(matrix):
     flat_list = []
     for row in matrix:
         flat_list.extend(row)
     return flat_list

In [4]:
from pyepo.model.grb import optGrbModel
num_item = 5*n_periods # number of predictions (Forward bid and Hydrogen)

# optimization model
class hydrogenPlanning_2(optGrbModel):
    def __init__(self, realized, *args, **kwargs):        
        #Fixed parameters
        self.max_elec = 10
        self.max_wind = 10
        self.nominal_wind = 10
        self.min_production = 50
        self.periods = np.arange(len(realized))
        self.E_real = realized
        super().__init__()

    def _getModel(self):

        self.initial_plan = Model("Gurobi.Optimizer")

        # Definition of variables
        self.var = self.initial_plan.addVars((5*len(self.periods)), name="x")
        # 1-24: Hydrogen plan, 25-48: Forward bids, 49-72: Up regulation, 73-96: Down regulation
        #97-120: Hydrogen extra
        # Objective: Maximize profit
        self.initial_plan.modelSense = GRB.MAXIMIZE

        # Constraints
        # Max capacity
        self.initial_plan.addConstr(self.min_production <= gp.quicksum(self.var[t] for t in self.periods), name="min_hydrogen_production")
        for t in np.arange(0,len(self.periods)):
            self.initial_plan.addConstr(self.var[t] >= 0, name=f"elec_capacity_lb_{t}")
            self.initial_plan.addConstr(self.var[t] <= self.max_elec, name=f"elec_capacity_ub_{t}")
        for t in np.arange(len(self.periods),2*len(self.periods)):
            self.initial_plan.addConstr(self.var[t] >= -self.max_elec, name=f"wind_capacity_lb_{t}")
            self.initial_plan.addConstr(self.var[t] <= self.max_wind, name=f"wind_capacity_ub_{t}")
        for t in np.arange(2*len(self.periods),3*len(self.periods)):
            self.initial_plan.addConstr(self.var[t] >= 0, name=f"up_regulation_lb_{t}")
            self.initial_plan.addConstr(self.var[t] <= 10*self.max_wind, name=f"up_regulation_ub_{t}")
        for t in np.arange(3*len(self.periods),4*len(self.periods)):
            self.initial_plan.addConstr(self.var[t] >= 0, name=f"dw_regulation_lb_{t}")
            self.initial_plan.addConstr(self.var[t] <= 10*self.max_wind, name=f"dw_regulation_ub_{t}")

        #notsure about this one
        for t in np.arange(0,len(self.periods)):
            self.initial_plan.addConstr(self.var[t] + self.var[t+24] <= self.max_wind, name=f"{t}")
        #added constraints for extra hydrogen
        for t in np.arange(4*len(self.periods),5*len(self.periods)):
            self.initial_plan.addConstr(self.var[t] >= -self.max_elec, name=f"extra_hydrogen_lb_{t}")
            self.initial_plan.addConstr(self.var[t] <= self.max_elec, name=f"extra_hydrogen_ub_{t}")
       #Balance constraint 
        for t in np.arange(0,len(self.periods)):
            self.initial_plan.addConstr(self.E_real[t] - self.var[t] - self.var[t+24] == -self.var[t+48] + self.var[t+72] + self.var[t+96], name=f"balancing_{t}")
            #initial_plan.addConstr(-x[0,t] + self.min_production/len(self.periods) - x[4,t] <= 0, name=f"slack_{t}")


        # Reference point is from extra hydrogen, so initial has to be tt-24*4
        for ix, t in enumerate(np.arange(4*len(self.periods),5*len(self.periods))):
            if ix == 0:
                # Must not reduce below min production
                self.initial_plan.addConstr(self.var[t] >= 
                                            - (gp.quicksum(self.var[tt] for tt in np.arange(ix,len(self.periods))) - self.min_production), "c1")
            else:
                # Must not reduce below min production - can do if we have produced more than min production earlier
                self.initial_plan.addConstr(self.var[t] >= 
                                            - (gp.quicksum(self.var[tt-24*4] 
                                                           + self.var[tt] for tt in np.arange(4*len(self.periods),t-1)) +
                                          gp.quicksum(self.var[tt-24*4] for tt in np.arange(t,5*len(self.periods))) 
                                          - self.min_production), "c2")
            # Cannot produce more than max capacity:
            self.initial_plan.addConstr(self.var[t] + self.var[t-24*4] <= self.max_elec, "Extra hydrogen production capacity")
            self.initial_plan.addConstr(self.var[t] + self.var[t-24*4] >= 0, "Extra hydrogen production capacity")        
            """"
            how to implement this? without cost or objective function?
            if lambda_H < price_DW[t,s]
                @constraint(SAA, EH_extra[t,s] <= 0)
            end
            """
        
        return self.initial_plan, self.var
    
    def setObjective(self, c):
        # Objective: Maximize profit
        self.initial_plan.setObjective(gp.quicksum(self.var[t]*c[t] for t in np.arange(0,5*len(self.periods))), GRB.MAXIMIZE)
        for t in np.arange(4*len(self.periods),5*len(self.periods)):
            if c[t] < c[t-24]: # if lambda_H[t] < price_DW[t]
                    self.initial_plan.addConstr(self.var[t] <= 0)

    def get_plan(self):
        self.initial_plan.optimize()
        self.initial_plan.update()
        x_values = []
        for var in self.initial_plan.getVars():
            x_values.append(var.x)
        hydrogen = x_values[0:len(self.periods)]
        forward_bids = x_values[len(self.periods):2*len(self.periods)]
        return forward_bids, hydrogen


In [5]:
from matplotlib import pyplot as plt

def visLearningCurve(loss_log, loss_log_regret):
    # create figure and subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4))

    # draw plot for training loss
    ax1.plot(loss_log, color="c", lw=1)
    ax1.tick_params(axis="both", which="major", labelsize=12)
    ax1.set_xlabel("Iters", fontsize=16)
    ax1.set_ylabel("Loss", fontsize=16)
    ax1.set_title("Learning Curve on Training Set", fontsize=16)

    # draw plot for regret on test
    ax2.plot(loss_log_regret, color="royalblue", ls="--", alpha=0.7, lw=1)
    ax2.set_xticks(range(0, len(loss_log_regret), 2))
    ax2.tick_params(axis="both", which="major", labelsize=12)
    ax2.set_ylim(0, 1)
    ax2.set_xlabel("Epochs", fontsize=16)
    ax2.set_ylabel("Regret", fontsize=16)
    ax2.set_title("Learning Curve on Test Set", fontsize=16)

    plt.show()

In [6]:


wind_train = np.asarray([flatten_extend([realized[d:d+n_periods]]) for d in range(int(n_hours/n_periods))])
#wind_val   = wind_train[-n_val_days:,:]
#wind_train   = wind_train[:(n_days-n_val_days),:]
wind_test = np.asarray([flatten_extend([realized[d:d+n_periods]]) for d in range(int(n_hours/n_periods), int(2*n_hours/n_periods))])
"""
x_train = np.asarray([flatten_extend(features.values[d:d+n_periods]) for d in range(int(n_hours/n_periods))])
x_test = np.asarray([flatten_extend(features.values[d:d+n_periods]) for d in range(int(n_hours/n_periods), int(2*n_hours/n_periods))])
"""


x_train_df = features.iloc[:n_hours]

#x_train_df = features.iloc[:(n_hours-n_hours_val)]
#x_val_df = features.iloc[(n_hours-n_hours_val):n_hours]
x_test_df = features.iloc[n_hours:(n_hours+n_hours)]


# Create a StandardScaler object (fitted on train data)
scaler = StandardScaler()
scaler.fit(x_train_df)

# Standardize train and test dataframes separately
x_train_df = pd.DataFrame(scaler.transform(x_train_df), columns=x_train_df.columns)
#x_val_df = pd.DataFrame(scaler.transform(x_val_df), columns=x_val_df.columns)
x_test_df = pd.DataFrame(scaler.transform(x_test_df), columns=x_test_df.columns)


x_train = []
#x_val = []
#for i in range(0, len(x_val_df), 24):
#    x_val.append((x_val_df.iloc[i:i+24]).values.T.flatten())
x_test = []
for i in range(0, len(x_train_df), 24):
    x_train.append((x_train_df.iloc[i:i+24]).values.T.flatten())  # Extract 24 rows for each day

for i in range(0, len(x_test_df), 24):
    x_test.append((x_test_df.iloc[i:i+24]).values.T.flatten()) 


# Standardize x_train and x_test
#train_mean = np.mean(x_train, axis=0)
#train_std = np.std(x_train, axis=0)
#x_train_stand = (x_train - train_mean) / train_std
#x_test_stand = (x_test - train_mean) / train_std

c_train = np.asarray([flatten_extend([lambda_H_list, prices_F[d: d+n_periods], -prices_UP[d: d+n_periods], prices_DW[d: d+n_periods],lambda_H_list]) for d in range(int(n_hours/n_periods))])
#c_val   = c_train[-n_val_days:,:]
#c_train   = c_train[:(n_days-n_val_days),:]
c_test = np.asarray([flatten_extend([lambda_H_list, prices_F[d: d+n_periods], -prices_UP[d: d+n_periods], prices_DW[d: d+n_periods],lambda_H_list]) for d in range(int(n_hours/n_periods), int(2*n_hours/n_periods))])


In [7]:
from pyepo.data.datasetParams import optDatasetParams
dataset_train = optDatasetParams(hydrogenPlanning_2, x_train, c_train, wind_train)
#dataset_val = optDatasetParams(hydrogenPlanning, x_val, c_val, wind_val)
dataset_test = optDatasetParams(hydrogenPlanning_2, x_test, c_test, wind_test)

batch_size = 1
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False)
#loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

Optimizing for optDataset...


  0%|          | 0/50 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-10


100%|██████████| 50/50 [00:00<00:00, 81.71it/s]


Optimizing for optDataset...


100%|██████████| 50/50 [00:00<00:00, 71.18it/s]


In [8]:
# prediction model
from torch import nn

# Initialize the model
constant_indices = np.zeros(num_item)
constant_indices[0:24] = 1
constant_indices[num_item-24:num_item] = 1
grad_mask = (constant_indices == 0).astype(int)

# prediction model
class LinearRegression(nn.Module):

    def __init__(self, input_size, output_size, neurons, dropout, constant_indices, constant_values, mask):
        super(LinearRegression, self).__init__()
        #self.linear = nn.Linear(num_feat, num_item)
        self.linear = nn.Sequential( 
            nn.Linear(input_size, neurons),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(neurons, neurons),
            nn.ReLU(),
            nn.Linear(neurons, output_size)
        )
        self.constant_indices = constant_indices
        self.constant_values = constant_values
        self.mask = mask

    def forward(self, x):
        out = self.linear(x)
        for i in range(len(self.constant_indices)):
            if self.constant_indices[i] == 1:
                if i < 24:
                    out[:,i] = self.constant_values[0]
                else:
                    out[:,i] = self.constant_values[1]
        return out


Optimize for regret, which will have to be done on validation set 

In [13]:
def trainModel(config=None):#, num_epochs=20, lr=1e-2):
    # set adam optimizer
    with wandb.init(config=config):
        config = wandb.config
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        #pprint(config)
        reg = LinearRegression(num_feat, num_item, config.neurons, config.dropout, constant_indices=constant_indices, constant_values=[price_H, price_H], mask=grad_mask)
        # cuda
        if torch.cuda.is_available():
            reg = reg.cuda()
        # init SPO+ loss
        spop = pyepo.func.SPOPlus
    
        optimizer = torch.optim.Adam(reg.parameters(), lr=config.lr)
        lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = config.gamma)
        # train mode
        reg.train()
        # init log
        loss_log = []
        # using validation regret instead of test regret
        loss_log_regret = [regretParams(reg, hydrogenPlanning_2, loader_test, wind_test)]
        # init elpased time
        elapsed = 0
        wandb.watch(reg, log_freq=100)
        for epoch in range(config.num_epochs):
            # start timing
            tick = time.time()
            # load data
            for i, data in enumerate(tqdm(loader_train)):
                wind = wind_train[i]
                opt_model = hydrogenPlanning_2(wind)
                loss_func = spop(opt_model, processes=1)
                x, c, w, z = data
                # cuda
                if torch.cuda.is_available():
                    x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
                # forward pass
                cp = reg(x)
                if config.method_name == "spo+":
                    loss = loss_func(cp, c, w, z)
                if config.method_name in ["ptb", "pfy", "imle", "nce", "cmap"]:
                    loss = loss_func(cp, w)
                if config.method_name in ["dbb", "nid"]:
                    loss = loss_func(cp, c, z)
                if config.method_name == "ltr":
                    loss = loss_func(cp, c)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                #for param in reg.parameters():
                #    if param.grad is not None:
                #        param.grad = (param.grad.T * reg.mask).T.type(torch.FloatTensor)
                #for name, param in reg.named_parameters():
                #    wandb.log({f"{name}.grad": param.grad.norm()}, step=epoch)
                optimizer.step()
                # record time
                tock = time.time()
                elapsed += tock - tick
                # log
                loss_log.append(loss.item())
                wandb.log({"Linear loss": loss})
            lr_scheduler.step()
            # validation regret
            regret = regretParams(reg, hydrogenPlanning_2, loader_test, wind_test)
            loss_log_regret.append(regret)
            wandb.log({"Regret": regret})
            print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, loss.item(), regret*100))
      
        print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))
        return reg, loss_log, loss_log_regret

In [14]:
# Hyper parameters

import pprint
sweep_config = {
    'method': 'random', # grid, random
    'metric': {
      'name': 'Regret',
      'goal': 'minimize'   
    },
}
parameters_dict =  {
        'lr': {
            'values': [1e-2, 1e-3, 1e-4]
        },
        'gamma': {
            'values': [0.9, 0.95, 0.99]
        },
        'num_epochs': {
            'values': [10,20]
        },
        'neurons': {
            'values': [32, 64, 128]
            },
            'dropout': {
                'values': [0.1, 0.2, 0.3]
            },
            #"loss_function": {"value":spop},
            "method_name": {"value":"spo+"},
    }
sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config, entity="Pyepo_special",project="Sweep Pyepo")

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'Regret'},
 'parameters': {'dropout': {'values': [0.1, 0.2, 0.3]},
                'gamma': {'values': [0.9, 0.95, 0.99]},
                'lr': {'values': [0.01, 0.001, 0.0001]},
                'method_name': {'value': 'spo+'},
                'neurons': {'values': [32, 64, 128]},
                'num_epochs': {'values': [10, 20]}}}
Create sweep with ID: h36gl3an
Sweep URL: https://wandb.ai/Pyepo_special/Sweep%20Pyepo/sweeps/h36gl3an


In [15]:
#loss_log, loss_log_regret = trainModel(loss_function=spop, method_name="spo+")
wandb.agent(sweep_id, function=trainModel,count=2)

wandb: Agent Starting Run: 1a9gfa2m with config:
wandb: 	dropout: 0.1
wandb: 	gamma: 0.9
wandb: 	lr: 0.01
wandb: 	method_name: spo+
wandb: 	neurons: 64
wandb: 	num_epochs: 10
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


100%|██████████| 50/50 [00:00<00:00, 65.57it/s]


Epoch  1,  Loss: 27190.9375,  Regret: 52.0185%


100%|██████████| 50/50 [00:00<00:00, 66.32it/s]


Epoch  2,  Loss: 28921.9395,  Regret: 51.9295%


...

100%|██████████| 50/50 [00:01<00:00, 47.83it/s]


Epoch  9,  Loss: 19390.4121,  Regret: 53.7761%


100%|██████████| 50/50 [00:00<00:00, 51.07it/s]


Epoch 10,  Loss: 17979.7480,  Regret: 53.9431%
Total Elapsed Time: 347.66 Sec.


Linear loss,▄█▆▆▄▇▃▆▅▆▃▄▆▆▃▅▅▂▄▄▃▅▂▇▆▁▃▃▂▂▂▇▇▁▂▄▁▁▂▆
Regret,▁▁▃▃▃▄█▆▇█
Linear loss,17979.74805
Regret,0.53943


In [16]:
# Redo training with best hyperparameters
best_config={
        'lr': 1e-2,
        'gamma': 0.9,
        'num_epochs': 10,
        'neurons': 40,
        'dropout': 0.5,
        "method_name": "spo+",
    }

reg, loss_log, loss_log_regret = trainModel(best_config)

100%|██████████| 50/50 [00:00<00:00, 62.36it/s]


Epoch  1,  Loss: 24898.6602,  Regret: 51.8141%


100%|██████████| 50/50 [00:00<00:00, 63.32it/s]


Epoch  2,  Loss: 24210.2676,  Regret: 51.9260%


100%|██████████| 50/50 [00:01<00:00, 47.64it/s]


...

100%|██████████| 50/50 [00:00<00:00, 55.47it/s]


Epoch  9,  Loss: 18369.9102,  Regret: 53.0118%


100%|██████████| 50/50 [00:00<00:00, 55.95it/s]


Epoch 10,  Loss: 18460.1660,  Regret: 53.3072%
Total Elapsed Time: 332.68 Sec.


Linear loss,▄█▅▇▄▇▃▆▄▆▃▄▅▆▂▅▅▂▃▄▃▅▂▇█▁▃▃▂▂▃▆▇▁▃▅▂▁▂▆
Regret,▁▂▃▅▂▆▆▇▇█
Linear loss,18460.16602
Regret,0.53307


Try on test set 

In [17]:
forward_bids = []
hydrogen_plan = []
reg.eval()
for i, data in enumerate(loader_test):
    x, c, w, z = data
    if torch.cuda.is_available():
        x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
    predicted_costs = reg(x).detach().numpy()[0]
    model = hydrogenPlanning_2(realized=wind_test[i])
    model.setObjective(predicted_costs)
    forward, hydrogen = model.get_plan()
    forward_bids.extend(forward)
    hydrogen_plan.extend(hydrogen)


In [18]:
pd.DataFrame({"forward bid" : forward_bids,"hydrogen production" : hydrogen_plan}).to_csv("ILO_base.csv", index=False)